# Generative Model 기반 Data Augmentation 실험

## 목차
1. 환경 설정 및 데이터 준비
2. 목적1: 최적 생성모델 탐색
3. 목적2: 전통적 증강 포화점 및 하이브리드 증강
4. 결과 분석 및 시각화

## 1. 환경 설정 및 데이터 준비

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json

from config import *
from data import CIFAR10FewShot
from utils import set_seed

set_seed(SEED)

print(f"Device: {DEVICE}")
print(f"Num GPUs: {NUM_GPUS}")
print(f"PyTorch version: {torch.__version__}")

import warnings
warnings.filterwarnings('ignore')

Device: cuda
Num GPUs: 1
PyTorch version: 2.8.0+cu129


In [2]:
# Few-shot 데이터 준비
print("Preparing CIFAR-10 few-shot dataset...")
few_shot = CIFAR10FewShot(samples_per_class=100, seed=SEED)

print("\nClass distribution:")
distribution = few_shot.get_class_distribution()
for cls, cnt in distribution.items():
    print(f"  Class {cls}: {cnt} samples")

print(f"\nTotal few-shot samples: {len(few_shot.few_shot_indices)}")
print(f"Test set size: {len(few_shot.test_dataset)}")

Preparing CIFAR-10 few-shot dataset...
Few-shot indices saved to /home/nill/Jeongbin/ML_TP/data/few_shot_indices_100shot.pkl

Class distribution:
  Class 0: 100 samples
  Class 1: 100 samples
  Class 2: 100 samples
  Class 3: 100 samples
  Class 4: 100 samples
  Class 5: 100 samples
  Class 6: 100 samples
  Class 7: 100 samples
  Class 8: 100 samples
  Class 9: 100 samples

Total few-shot samples: 1000
Test set size: 10000


### Diffusion 모델 로드

**ImageNet 64x64** pretrained diffusion 모델을 사용합니다.

- CIFAR-10 이미지(32x32)는 학습 시 64x64로 업샘플링됩니다.
- 생성된 이미지(64x64)는 32x32로 다운샘플링됩니다.

In [3]:
from diffusers import UNet2DModel, DDPMScheduler

print("Loading ImageNet 64x64 diffusion model...")

# ImageNet 64x64 Consistency Model - ImageNet으로 사전학습된 모델
model_id = "openai/diffusers-ct_imagenet64"

base_model = UNet2DModel.from_pretrained(
    model_id,
    subfolder="unet"  # UNet만 로드
)
base_model.eval()
base_model.to(DEVICE)

# Scheduler
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

print("\nDiffusion model loaded successfully.")
print(f"Model size: 64x64")
print(f"Parameters: {sum(p.numel() for p in base_model.parameters()):,}")
print("\nNote:")
print("  - CIFAR-10 images (32x32) will be upsampled to 64x64 for training")
print("  - Generated images (64x64) will be downsampled to 32x32 for classification")

2025-11-22 00:46:15.473482: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-22 00:46:15.493762: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-22 00:46:16.028120: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Loading ImageNet 64x64 diffusion model...

Diffusion model loaded successfully.
Model size: 64x64
Parameters: 295,899,267

Note:
  - CIFAR-10 images (32x32) will be upsampled to 64x64 for training
  - Generated images (64x64) will be downsampled to 32x32 for classification


## 2. 목적1: 최적 생성모델 탐색

### 실험 개요
- 모델군 9개 구성 (Non-finetuned 1개 + LoRA 8개)
- Baseline 성능 측정
- 각 생성모델로 증강 및 분류 성능 비교
- FID 계산
- 최적 모델 선정

In [4]:
from experiment_optimalGen import OptimalGenExperiment

# 실험 초기화
exp1 = OptimalGenExperiment(
    base_diffusion_model=base_model,
    samples_per_class=100,
    seed=SEED
)

print("Objective 1 experiment initialized.")

Few-shot indices saved to /home/nill/Jeongbin/ML_TP/data/few_shot_indices_100shot.pkl
[2025-11-22 00:46:21] Original dataset: 1000 images
Objective 1 experiment initialized.


In [5]:
# 실험 실행
print("Starting Objective 1 experiment...")
print("This may take several hours to complete.")

results1 = exp1.run_full_experiment()

print("\nObjective 1 completed!")

Starting Objective 1 experiment...
This may take several hours to complete.
[2025-11-22 00:46:22] ============================================================
[2025-11-22 00:46:22] Objective 1: Optimal Generative Model Search
[2025-11-22 00:46:22] ============================================================
[2025-11-22 00:46:22] Preparing traditional augmentation (5x)...
Generating 5 augmentations per sample...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1442.08it/s]


[2025-11-22 00:46:23] Generated 5000 traditional augmented images
[2025-11-22 00:46:23] 
=== Baseline 1: Original data only ===
[2025-11-22 00:46:23] Starting training for 100 epochs
[2025-11-22 00:46:23] Train size: 1000
[2025-11-22 00:46:23] Test size: 10000


Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 186.58it/s]


[2025-11-22 00:46:24] Epoch 1/100 - Loss: 5.1288, Train Acc: 12.50%, Test Acc: 10.01%


...

LoRA Epoch 5: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [01:02<00:00,  6.00it/s, loss=5.7499]


[2025-11-22 04:04:56] Epoch 5/50 - Loss: 5.7499


LoRA Epoch 6: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [01:02<00:00,  6.01it/s, loss=3.5736]


[2025-11-22 04:05:58] Epoch 6/50 - Loss: 3.5736


LoRA Epoch 7:  53%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                                       | 199/375 [00:33<00:29,  6.01it/s, loss=2.1803]IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

/ATen/native/cuda/Indexing.cu:1478: indexSelectSmallIndex: block: [1,0,0], thread: [89,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1478: indexSelectSmallIndex: block: [1,0,0], thread: [90,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1478: indexSele

In [6]:
# Baseline 결과
print("=== Baseline Results ===")
for name, acc in results1['baselines'].items():
    print(f"{name}: {acc:.2f}%")

# 생성모델 결과
print("\n=== Generative Model Results ===")
for name, acc in results1['generative_models'].items():
    fid = results1['fid_scores'].get(name, 'N/A')
    print(f"{name}: {acc:.2f}% (FID: {fid})")

# 최적 모델
print("\n=== Best Model ===")
best = results1['best_model']
print(f"Model: {best['name']}")
print(f"Accuracy: {best['accuracy']:.2f}%")
print(f"FID: {best['fid']}")

=== Baseline Results ===


NameError: name 'results1' is not defined